<a href="https://colab.research.google.com/github/Amisha1019/Gen-AI-Customer-Service-Bot-Internship-Task/blob/main/Task_2_gen_Ai_internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
  !pip install -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement faiss-cpu==1.7.4 (from versions: 1.8.0, 1.8.0.post1, 1.9.0, 1.9.0.post1, 1.10.0, 1.11.0, 1.11.0.post1, 1.12.0)
ERROR: No matching distribution found for faiss-cpu==1.7.4


In [3]:
  GOOGLE_API_KEY="your_api_key_here"

In [6]:
import base64
import io
from typing import Optional
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse, StreamingResponse
from pydantic import BaseModel
from PIL import Image
import httpx
from dotenv import load_dotenv
import os


load_dotenv()


API_KEY = os.getenv("GOOGLE_API_KEY")
PALM_BASE = os.getenv("PALM_API_BASE", "https://generativelanguage.googleapis.com/v1")
GEMINI_BASE = os.getenv("GEMINI_API_BASE", "https://api.generative.google/v1")
PROJECT_ID = os.getenv("PROJECT_ID")


app = FastAPI(title="Multimodal Chatbot — PaLM + Gemini")

In [8]:
def google_headers():
    return {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}


async def call_palm_text(prompt: str, max_tokens: int = 512):
    # Example using PaLM text generation endpoint
    url = f"{PALM_BASE}/models/text-bison:generate"
    payload = {
        "prompt": {"text": prompt},
        "temperature": 0.7,
        "maxOutputTokens": max_tokens
    }
    async with httpx.AsyncClient(timeout=30.0) as client:
        r = await client.post(url, json=payload, headers=google_headers())
        r.raise_for_status()
        return r.json()


async def call_gemini_image_generate(prompt: str, size: str = "1024x1024"):
    # Example Gemini image generation endpoint -- placeholder shape
    url = f"{GEMINI_BASE}/images:generate"
    payload = {"prompt": prompt, "size": size, "num_images": 1}
    async with httpx.AsyncClient(timeout=60.0) as client:
        r = await client.post(url, json=payload, headers=google_headers())
        r.raise_for_status()
        return r.json()


async def call_gemini_vision_analyze(image_bytes: bytes):
    # Example of sending image bytes encoded as base64 to Gemini Vision/Multimodal endpoint
    url = f"{GEMINI_BASE}/vision:analyze"
    b64 = base64.b64encode(image_bytes).decode("utf-8")
    payload = {"image": {"content": b64}, "features": ["TEXT_DETECTION", "LABEL_DETECTION", "OBJECT_DETECTION"]}
    async with httpx.AsyncClient(timeout=30.0) as client:
        r = await client.post(url, json=payload, headers=google_headers())
        r.raise_for_status()
        return r.json()

In [10]:
class TextRequest(BaseModel):
    prompt: str


class ImageGenRequest(BaseModel):
    prompt: str
    size: Optional[str] = "1024x1024"

In [17]:
@app.post('/content/text_to_image_dataset.csv')
async def generate_text(req: TextRequest):
    """Generate a text reply using PaLM"""
    try:
        resp = await call_palm_text(req.prompt)
        # Response shape may vary — adapt to your API version
        output_text = resp.get('candidates', [{}])[0].get('content') or resp.get('output') or str(resp)
        return JSONResponse({"text": output_text, "raw": resp})
    except httpx.HTTPError as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


@app.post('/content/text_to_image_dataset.csv')
async def generate_image(req: ImageGenRequest):
    """Generate an image using Gemini Image API and return base64.
    In production you'd return a URL to the stored image.
    """
    try:
        resp = await call_gemini_image_generate(req.prompt, req.size)
        # adapt according to actual Gemini result shape. Many providers return base64 image(s).
        images = []
        # Try common shapes: 'images' -> list of {b64: '...'} or 'data' -> ...
        if isinstance(resp, dict):
            if 'images' in resp:
                images = resp['images']
            elif 'data' in resp and isinstance(resp['data'], list):
                images = resp['data']
            else:
                images = [resp]
        return JSONResponse({"images": images, "raw": resp})
    except httpx.HTTPError as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


@app.post('/vision/analyze')
async def vision_analyze(file: UploadFile = File(...)):
    """Accept an uploaded image and return Gemini Vision analysis."""
    try:
        body = await file.read()
        resp = await call_gemini_vision_analyze(body)
        return JSONResponse({"analysis": resp})
    except httpx.HTTPError as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


# Simple health endpoint
@app.get('/health')
async def health():
    return {"status": "ok"}

In [41]:
import requests
import base64
from PIL import Image
from io import BytesIO
import json

# ---- Configuration ----
BACKEND_URL = "https://upload.wikimedia.org/wikipedia/commons/e/e3/Tropical_Beach.jpg"   # or your deployed FastAPI server

# ------------------------
def generate_text(prompt):
    """
    Send text prompt to FastAPI backend -> returns generated text.
    """
    url = f"{BACKEND_URL}https://upload.wikimedia.org/wikipedia/commons/e/e3/Tropical_Beach.jpg"
    payload = {"prompt": prompt}
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        print("Text Generation Successful!")
        return data.get("text", "")
    except Exception as e:
        print("Error generating text:", e)
        return None


def generate_image_from_prompt(prompt, size="1024x1024"):
    """
    Generate an image from text prompt using the /generate/image endpoint.
    Saves and shows the image locally.
    """
    url = f"{BACKEND_URL}/content/text_to_image_dataset.csv"
    payload = {"prompt": prompt, "size": size}
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()

        # Extract image
        image_entry = None
        if "images" in data and len(data["images"]) > 0:
            image_entry = data["images"][0]

        if not image_entry:
            print("No image found in response.")
            return None

        b64 = (
            image_entry.get("b64")
            or image_entry.get("base64")
            or image_entry.get("data")
            or image_entry.get("content")
        )

        if b64:
            img_data = base64.b64decode(b64)
            image = Image.open(BytesIO(img_data))
            image.save("generated_image.png")
            image.show()
            print("Image saved as generated_image.png")
            return "generated_image.png"
        elif "url" in image_entry:
            print("Image available at URL:", image_entry["https://upload.wikimedia.org/wikipedia/commons/e/e3/Tropical_Beach.jpg"])
            return image_entry["https://upload.wikimedia.org/wikipedia/commons/e/e3/Tropical_Beach.jpg"]
        else:
            print("Could not decode image data.")
            return None

    except Exception as e:
        print("Error generating image:", e)
        return None


def analyze_image(image_path):
    """
    Upload an image to the backend for visual analysis.
    """
    url = f"{BACKEND_URL}/vision/analyze"
    try:
        with open(image_path, "rb") as f:
            files = {"file": "/content/text_to_image_dataset.csv"}
            response = requests.post(url, files=files)
        response.raise_for_status()
        data = response.json()
        print("Image analysis complete!")
        print(json.dumps(data.get("analysis", {}), indent=2))
        return data.get("analysis", {})
    except Exception as e:
        print("Error analyzing image:", e)
        return None


# ------------------------
# Example Usage:
if __name__ == "__main__":
    # Generate text
    text_prompt = "Describe the beauty of space exploration."
    text_output = generate_text(text_prompt)
    print("/content/text_to_image_dataset.csv", text_output)

    # Generate an image
    image_prompt = "A futuristic city on Mars at sunset."
    image_path = generate_image_from_prompt(image_prompt)

    # Analyze an existing image (optional)
    if image_path:
        analyze_image("/content/text_to_image_dataset.csv")


Error generating text: 405 Client Error: Method Not Allowed for url: https://upload.wikimedia.org/wikipedia/commons/e/e3/Tropical_Beach.jpghttps://upload.wikimedia.org/wikipedia/commons/e/e3/Tropical_Beach.jpg
/content/text_to_image_dataset.csv None
Error generating image: 405 Client Error: Method Not Allowed for url: https://upload.wikimedia.org/wikipedia/commons/e/e3/Tropical_Beach.jpg/content/text_to_image_dataset.csv
